In [1]:
# Import PuLP modeler functions
import pulp
from pulp import LpProblem, LpMaximize
from pulp import LpVariable, LpStatus

In [2]:
pulp.__version__

'2.7.0'

# PuLP

PuLP é uma biblioteca Python para resolver problemas de programação linear.
Esta biblioteca é realmente útil para escrever o modelo do problema de otimização.
Por exemplo, ela nao tem as mesmas limitacoes que vimos para linprog() do SciPy.
PuLP inclui um algoritmo de resolução padrão para esses problemas baseado em simplex combinado com outros algoritmos, mas permite que você use outros solucionadores (GUROBI, GLPK, CPLEX, etc.).

Vamos revisitar o problema que apresentamos no notebook [01-Alocacao de recursos.ipynb](https://github.com/h3dema/linear-programming/blob/main/01-Alocacao%20de%20recursos.ipynb).



### Recordando

O problema possui somente duas variaveis e o sistema de equações lineares é:


\begin{matrix}
maximize~z = & x + 2y \\
sujeito~a & 2x + y \leq 20 \\
 & -4x + 5y \leq 10 \\
 & -x + 2y \ge -2 \\
 & -x + 5y = 15 \\
 & x \ge 0 \\
 & y \ge 0
\end{matrix}


In [3]:
# definimos o problema como de maximizacão
prob = LpProblem("Problema_simples", LpMaximize)

In [4]:
# criamos as variáveis
x = LpVariable("x", lowBound=0)
y = LpVariable("y", 0)  # o segundo parametro é lowBound !

In [5]:
# funcão objetivo
prob += x + 2 * y  # nosso objetivo

In [6]:
prob += 2 * x + y <= 20  # Eq. 1

In [7]:
prob += -4 * x + 5 * y <= 10  # Eq. 2
prob += -x + 2 * y >= -2  # Eq. 3
prob += -x + 5 * y == 5  # Eq. 4

Note:
1. como escrevemos o sinal de igual na Eq. 4 acima. O sinal é **==**
2. as restricões sao adicionadas ao problema `prob` usando **+=**

In [8]:
# podemos gravar o problema em um arquivo
prob.writeLP("Problema_Simples.lp")

[x, y]

In [10]:
# O problema é resolvido utilizando um solver selecionado por PuLP
# já que não indicamos aquele que desejamso
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/idlab/linear-programming/.venv/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/2e0ccd84280043aca2a410b84ac5d6c4-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/2e0ccd84280043aca2a410b84ac5d6c4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 20 RHS
At line 25 BOUNDS
At line 26 ENDATA
Problem MODEL has 4 rows, 2 columns and 8 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-4) rows, 0 (-2) columns and 0 (-8) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 11.333333
After Postsolve, objective 11.333333, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 11.33333333 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallc

1

In [12]:
# Podemos obter o status
print("Status:", LpStatus[prob.status])

# e os valores ótimos para cada variável de decisão
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
# e o valor final da funcão objetivo
print("F.obj. = ", pulp.value(prob.objective))

Status: Optimal
x = 6.6666667
y = 2.3333333
F.obj. =  11.3333333


***

In [13]:
help(LpVariable)

Help on class LpVariable in module pulp.pulp:

class LpVariable(LpElement)
 |  LpVariable(name, lowBound=None, upBound=None, cat='Continuous', e=None)
 |  
 |  This class models an LP Variable with the specified associated parameters
 |  
 |  :param name: The name of the variable used in the output .lp file
 |  :param lowBound: The lower bound on this variable's range.
 |      Default is negative infinity
 |  :param upBound: The upper bound on this variable's range.
 |      Default is positive infinity
 |  :param cat: The category this variable is in, Integer, Binary or
 |      Continuous(default)
 |  :param e: Used for column based modelling: relates to the variable's
 |      existence in the objective function and constraints
 |  
 |  Method resolution order:
 |      LpVariable
 |      LpElement
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name, lowBound=None, upBound=None, cat='Continuous', e=None)
 |      Initialize self.  See help(type(self)) for 

In [14]:
help(prob.solve)

Help on method solve in module pulp.pulp:

solve(solver=None, **kwargs) method of pulp.pulp.LpProblem instance
    Solve the given Lp problem.
    
    This function changes the problem to make it suitable for solving
    then calls the solver.actualSolve() method to find the solution
    
    :param solver:  Optional: the specific solver to be used, defaults to the
          default solver.
    
    Side Effects:
        - The attributes of the problem object are changed in
          :meth:`~pulp.solver.LpSolver.actualSolve()` to reflect the Lp solution



In [16]:
solver_list = pulp.listSolvers(onlyAvailable=True)
solver_list

['GLPK_CMD', 'PULP_CBC_CMD', 'COIN_CMD']

In [17]:
# problema de maximizacão
prob = LpProblem("Problema_simples", LpMaximize)

# criamos as variáveis
x = LpVariable("x", lowBound=0, cat="Integer")  # <---- agora é inteiro
y = LpVariable("y", lowBound=0, cat="Integer")  # <---- agora é inteiro

# funcão objetivo
prob += x + 2 * y  # nosso objetivo
prob += 2 * x + y <= 20  # Eq. 1
prob += -4 * x + 5 * y <= 10  # Eq. 2
prob += -x + 2 * y >= -2  # Eq. 3
prob += -x + 5 * y == 5  # Eq. 4

# vamos definir também um solver
solver = pulp.COIN_CMD(msg=True, warmStart=True)
prob.solve(solver=solver)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Mar 24 2020 

command line - cbc /tmp/76902670c8264b178c30979b7ffd2ccb-pulp.mps max mips /tmp/76902670c8264b178c30979b7ffd2ccb-pulp.mst timeMode elapsed branch printingOptions all solution /tmp/76902670c8264b178c30979b7ffd2ccb-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 24 RHS
At line 29 BOUNDS
At line 32 ENDATA
Problem MODEL has 4 rows, 2 columns and 8 elements
Coin0008I MODEL read with 0 errors
opening mipstart file /tmp/76902670c8264b178c30979b7ffd2ccb-pulp.mst.
MIPStart values read for 2 variables.
Option for timeMode changed from cpu to elapsed
Continuous objective value is 11.3333 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -9 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active afte

1

In [19]:
# Podemos obter o status
print("Status:", LpStatus[prob.status])

# e os valores ótimos para cada variável de decisão
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
# e o valor final da funcão objetivo
print("F.obj. = ", pulp.value(prob.objective))

Status: Optimal
x = 5.0
y = 2.0
F.obj. =  9.0
